# Using spatial autocorrelation on CORONAVIRUS
- You need coronapy (see https://github.com/gisfromscratch/corona-py)

In [1]:
from corona_analytics import hopkins_client

corona_client = hopkins_client()
corona_data = corona_client.query_hopkins_data()
corona_data

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,84,94,110,110,120,170,174,237,273,281
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,146,174,186,197,212,223,243,259,277,304
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,302,367,409,454,511,584,716,847,986,1171
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,188,224,267,308,334,370,376,390,428,439
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,3,4,4,5,7,7,7,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,NaN,Botswana,-22.328500,24.684900,0,0,0,0,0,0,...,0,0,0,0,0,3,4,4,4,4
254,NaN,Burundi,-3.373100,29.918900,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,3,3
255,NaN,Sierra Leone,8.460555,-11.779889,0,0,0,0,0,0,...,0,0,0,0,0,0,1,2,2,2
256,"Bonaire, Sint Eustatius and Saba",Netherlands,12.178400,-68.238500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2


In [33]:
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry, Point, SpatialReference
from arcgis.geometry.filters import intersects

def query_administrative_layer(latitude, longitude):
    admin_layer = FeatureLayer("https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/World_Administrative_Divisions/FeatureServer/0")
    location = Point()
    location.x = longitude
    location.y = latitude
    location.spatialReference = SpatialReference(4326)
    intersect_filter = intersects(location, sr=location.spatialReference)
    admin_result = admin_layer.query(geometry_filter=intersect_filter)
    return admin_result.features

In [34]:
query_administrative_layer(33, 65)

[{"geometry": {"rings": [[[7270324.91811174, 3948653.42512339], [7270233.43249654, 3947211.45901057], [7270576.52327558, 3945998.51540654], [7270576.52327558, 3944671.28848752], [7270896.73604342, 3943916.21001573], [7270828.11522463, 3942909.51506803], [7270622.25276827, 3942680.73301748], [7270416.40362681, 3941994.39787717], [7270622.25276827, 3940827.76414113], [7271560.0529052, 3940461.78065024], [7273435.62644915, 3940416.03343554], [7273618.58436462, 3940187.30054634], [7273641.46237582, 3939661.21592763], [7274236.1716336, 3939043.70335221], [7273936.95246742, 3938654.89018123], [7274599.91102809, 3937557.19551312], [7274782.77603956, 3936185.187476], [7274508.45204272, 3935339.20639109], [7274645.62730599, 3935270.63183476], [7274622.76250959, 3935042.00028152], [7274394.16800543, 3934836.23565868], [7274005.52022681, 3933601.72455934], [7274256.99274216, 3933007.37708025], [7274256.99274216, 3932504.47553277], [7273502.6017258, 3931361.6504781], [7273113.99379179, 3931293.083